In [1]:
from pygame.image import load 
from pygame.surfarray import pixels_alpha
from pygame.transform import rotate

pygame 2.4.0 (SDL 2.26.4, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def load_images(sprites_path):
    base_image = load(sprites_path + 'base.png').convert_alpha()
    background_image = load(sprites_path + 'background-black.png').convert()
    pipe_images = [rotate(load(sprites_path + 'pipe-green.png').convert_alpha(), 180),load(sprites_path+'pipe-green.png').convert_alpha()]
    bird_images = [load(sprites_path + 'redbird-upflap.png').convert_alpha(),
                   load(sprites_path +
                        'redbird-midflap.png').convert_alpha(),
                    load(sprites_path + 'redbird-downflap.png').convert_alpha()]
    bird_hitmask = [pixels_alpha(image).astype(bool) for image in bird_images]
    pipe_hitmask = [pixels_alpha(image).astype(bool) for image in pipe_images]

    return base_image, background_image, pipe_images, bird_images, bird_hitmask, pipe_hitmask

In [3]:
from itertools import cycle
from random import randint
import pygame

In [4]:
pygame.init()

fps_clock = pygame.time.Clock()
fps = 30

In [5]:
screen_width = 288
screen_height = 512
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Flappy Bird')


In [6]:
base_image, background_image, pipe_images, bird_images, bird_hitmask, pipe_hitmask = load_images('sprites/')


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [7]:
bird_width = bird_images[0].get_width()
bird_height = bird_images[0].get_height()
pipe_width = pipe_images[0].get_width()
pipe_height = pipe_images[0].get_height()
pipe_gap_size = 100

In [8]:
bird_index_gen = cycle([0,1,2,1])



In [9]:
class FlappyBird(object):
    def __init__(self):
        self.pipe_vel_x = -4
        self.min_velocity_y = -8
        self.max_velocity_y = 10
        self.downward_speed = 1
        self.upward_speed = -9
        self.cur_velocity_y = 0
        self.iter = self.bird_index = self.score = 0 
        self.bird_x = int (screen_width / 5)
        self.bird_y = int ((screen_height - bird_height)/2)
        self.base_shift = base_image.get_width() - background_image.get_width()
        self.pipes = [self.gen_random_pipe(screen_width), self.gen_random_pipe(screen_width * 1.5)]
        self.is_flapped = False

    def check_collision(self):
        if bird_height + self.bird_y >= self.base_y -1 :
            return True
        
        bird_rect = pygame.Rect(self.bird_x, self.bird_y, bird_width , bird_height)

        for pipe in self.pipes:
            pipe_boxes = [pygame.Rect(pipe["x_upper"],
                                     pipe["y_upper"], pipe_width,  
                                     pipe_height ), pygame.Rect(pipe["x_lower"], 
                                     pipe["y_lower"], pipe_width, pipe_height )]

            
            if bird_rect.collidelist(pipe_boxes) == 1:
                return False
            
            for i in range(2):
                cropped_bbox = bird_rect.clip(pipe_boxes[i])
                x1 = cropped_bbox.x - bird_rect.x
                y1 = cropped_bbox.y - bird_rect.y
                x2 = cropped_bbox.x - pipe_boxes[i].x
                y2 = cropped_bbox.y - pipe_boxes[i].y

                for x in range(cropped_bbox.width):
                    for y in range(cropped_bbox.height):
                        if bird_hitmask[self.bird_index][x1 + x, y1+ y] and pipe_hitmask[i][x2+x, y2+y]:
                            return True
        

        return False


    def next_step(self, action):
        pygame.event.pump()

        reward = 0.1

        if action == 1:
            self.cur_velocity_y = self.upward_speed
            self.is_flapped = True

        bird_center_x = self.bird_x + bird_width / 2

        for pipe in self.pipes:
            pipe_center_x = pipe["x_upper"] + pipe_width/2

            if pipe_center_x < bird_center_x < pipe_center_x + 5:

                self.score += 1 
                reward = 1
                break

        
        if (self.iter + 1) % 3 ==0 :
            self.bird_index = next(bird_index_gen)

        self.iter = (self.iter + 1) % fps

        self.base_x = -((-self.base_x + 100) % self.base_shift)


        if self.curr_velocity_y < self.max_velocity_y and not self.is_flapped:

            self.curr_velocity_y += self.downward_speed

        self.is_flapped = False

        self.bird_y += min(self.cur_velocity_y,
                           self.bird_y - self.cur_velocity_y - bird_height)

        if self.bird_y < 0:
            self.bird_y = 0 


        for pipe in self.pipes:
            pipe["x_upper"] += self.pipe_vel_x
            pipe["x_lower"] += self.pipe_vel_x

        if 0 < self.pipes[0]["x_lower"] < 5:
            self.pipes.append(self.gen_random_pipe(
                screen_width + 10
            ))

        if self.pipes[0]["x_lower"] < -pipe_width:
            self.pipes.pop(0)

        if self.check_collision():
            is_done = True
            reward = -1
            self.__init__()
        else:
            is_done = False
            

        screen.blit(background_image, (0,0))
        screen.blit(base_image, (self.base_x, self.base_y))
        screen.blit(bird_images[self.bird_index], 
                    (self.bird_x , self.bird_y))

        for pipe in self.pipes:
            screen.blit(pipe_images[0], 
                        (pipe["x_upper"], pipe["y_upper"]))
            screen.blit(pipe_images[1],
                        (pipe["x_lower"], pipe["y_lower"]))

        image = pygame.surfarray.array3d(
            pygame.display.get_surface()
        )

        pygame.display.update()
        fps_clock.tick(fps)
        return image, reward , is_done



: 

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random

In [3]:
class DQNModel(nn.Module):
    def  __init__(self,n_action = 2):
        super(DQNModel, self).__init__()
        self.conv1 = nn.Conv2d(4,32, kernel_size=8, stride = 4)
        self.conv2 = nn.Conv2d(32,64, 4, stride = 2)
        self.conv3 = nn.Conv2d(64,64,3, stride=1)
        self.fc = nn.Linear(7*7*64, 512)
        self.out = nn.Linear(512, n_action)
        self._create_weights()


    def _create_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.uniform(m.weight, -0.01, 0.01)
                nn.init.constant_(m.bias, 0 )


    def forward(self,x):
        x= F.relu(self.conv1(x))
        x= F.relu(self.conv2(x))
        x= F.relu(self.conv3(x))
        x= F.view(x.size(0), -1)
        x= F.relu(self.fc(x))
        output = self.out(x)
        return output